In [3]:
import numpy as np
import pandas as pd

# from sklearn import preprocessing
# from sklearn.preprocessing import SimpleImputer
import statistics as stat


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.compose import ColumnTransformer
#from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier


from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt 

from datetime import timedelta

import seaborn as sns



from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


### reading files

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train = pd.read_csv("/content/drive/My Drive/train.csv")

In [0]:

valid = pd.read_csv("/content/drive/My Drive/test_QkPvNLx.csv")

### inspecting the data

In [14]:
print(train.shape, train.info())
print("----------------------------------------")
print(valid.shape, valid.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512087 entries, 0 to 512086
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ID                  512087 non-null  int64  
 1   Day_No              512087 non-null  int64  
 2   Course_ID           512087 non-null  int64  
 3   Course_Domain       512087 non-null  object 
 4   Course_Type         512087 non-null  object 
 5   Short_Promotion     512087 non-null  int64  
 6   Public_Holiday      512087 non-null  int64  
 7   Long_Promotion      512087 non-null  int64  
 8   User_Traffic        512087 non-null  int64  
 9   Competition_Metric  510323 non-null  float64
 10  Sales               512087 non-null  int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 43.0+ MB
(512087, 11) None
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36000 entries, 0 to 35999
Data columns (total 9 columns):
 #   Colu

### converting ID column as index column 

In [15]:
train.index_col = train.ID
valid.index_col = valid.ID
train.index_col

0              1
1              2
2              3
3              4
4              5
           ...  
512082    548023
512083    548024
512084    548025
512085    548026
512086    548027
Name: ID, Length: 512087, dtype: int64

In [0]:
print(train.shape, train.info())
print("----------------------------------------")
print(valid.shape, valid.info())

In [0]:
def inspect_data(data):
    return pd.DataFrame({"Data Type":data.dtypes,"No of Levels":data.apply(lambda x: x.nunique(),axis=0), "Levels":data.apply(lambda x: str(x.unique()),axis=0)})
inspectrain = inspect_data(train)
print(inspectrain)

In [0]:
inspecvalid = inspect_data(valid)
print(inspecvalid)

In [16]:
train.columns


Index(['ID', 'Day_No', 'Course_ID', 'Course_Domain', 'Course_Type',
       'Short_Promotion', 'Public_Holiday', 'Long_Promotion', 'User_Traffic',
       'Competition_Metric', 'Sales'],
      dtype='object')

In [17]:
#defining catagorical columns and numerical columns
cat_cols = ['Course_ID', 'Course_Domain', 'Course_Type','Short_Promotion','Public_Holiday','Long_Promotion']
num_cols = ['Competition_Metric','Sales',]

train[cat_cols] = train[cat_cols].apply(lambda x: x.astype('category'))
valid[cat_cols] = valid[cat_cols].apply(lambda x: x.astype('category'))
print(train.shape, train.info())
print("----------------------------------------")
print(valid.shape, valid.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512087 entries, 0 to 512086
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   ID                  512087 non-null  int64   
 1   Day_No              512087 non-null  int64   
 2   Course_ID           512087 non-null  category
 3   Course_Domain       512087 non-null  category
 4   Course_Type         512087 non-null  category
 5   Short_Promotion     512087 non-null  category
 6   Public_Holiday      512087 non-null  category
 7   Long_Promotion      512087 non-null  category
 8   User_Traffic        512087 non-null  int64   
 9   Competition_Metric  510323 non-null  float64 
 10  Sales               512087 non-null  int64   
dtypes: category(6), float64(1), int64(4)
memory usage: 23.0 MB
(512087, 11) None
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36000 entries, 0 to 35999
Data columns (total 9 colum

In [18]:
### Check how many null values are there in each of the columns 
print(train.isnull().sum())
print(valid.isnull().sum())

# subsetting rowswith null values
x = train[train['Competition_Metric'].isnull()]
y = valid[valid['Competition_Metric'].isnull()]
x = pd.DataFrame(x)
y = pd.DataFrame(y)
print(x.apply(lambda x: str(x.unique()),axis=0))
print(y.apply(lambda x: str(x.unique()),axis=0))




ID                       0
Day_No                   0
Course_ID                0
Course_Domain            0
Course_Type              0
Short_Promotion          0
Public_Holiday           0
Long_Promotion           0
User_Traffic             0
Competition_Metric    1764
Sales                    0
dtype: int64
ID                      0
Day_No                  0
Course_ID               0
Course_Domain           0
Course_Type             0
Short_Promotion         0
Public_Holiday          0
Long_Promotion          0
Competition_Metric    120
dtype: int64
ID                      [159515 159516 159517 ... 324022 324023 324024]
Day_No                [  1   2   3   4   5   6   7   8   9  10  11  ...
Course_ID                 [176, 354]\nCategories (2, int64): [176, 354]
Course_Domain         [Software Marketing, Development]\nCategories ...
Course_Type           [Course, Program]\nCategories (2, object): [Co...
Short_Promotion                   [0, 1]\nCategories (2, int64): [0, 1]
Public_Holi

In [0]:
x.columns

In [19]:
# subsetting data of mssing rows data to find out combination of course domain and course type
a = x[(x["Course_Domain"] == "Software Marketing")]
print(a.apply(lambda x: str(x.unique()),axis=0))
b = x[(x["Course_Domain"] == "Development")]
print(b.apply(lambda x: str(x.unique()),axis=0))

p = y[(y["Course_Domain"] == "Software Marketing")]
print(a.apply(lambda x: str(x.unique()),axis=0))
q = y[(y["Course_Domain"] == "Development")]
print(b.apply(lambda x: str(x.unique()),axis=0))


ID                    [159515 159516 159517 159518 159519 159520 159...
Day_No                [  1   2   3   4   5   6   7   8   9  10  11  ...
Course_ID                           [176]\nCategories (1, int64): [176]
Course_Domain         [Software Marketing]\nCategories (1, object): ...
Course_Type                  [Course]\nCategories (1, object): [Course]
Short_Promotion                   [0, 1]\nCategories (2, int64): [0, 1]
Public_Holiday                    [1, 0]\nCategories (2, int64): [1, 0]
Long_Promotion                          [0]\nCategories (1, int64): [0]
User_Traffic          [12180 18396 18291 19320 11592 16548 27636 192...
Competition_Metric                                                [nan]
Sales                 [120 138 132 139  95 117 237 167 166 153 143  ...
dtype: object
ID                    [323143 323144 323145 323146 323147 323148 323...
Day_No                [  1   2   3   4   5   6   7   8   9  10  11  ...
Course_ID                           [354]\nCategor

#Course_ID - 176, Course_Domain -Software Marketing, Course_Type - Course, Long_Promotion is 0}
#Course_ID - 354, Course_Domain -Development,Course_Type - Program, Long_Promotion is 0}
#impute these missing values with mean/ mode / median of  all courses with software marketing, course and with 0 long promotion for 176 
#impute these missing values with mean/ mode / median of  all courses with Development, Program and with 0 long promotion for 354

In [20]:
#finding mean/ mode / median of  all courses with software marketing, course and with 0 long promotion for 176
x = train.loc[(train["Course_Domain"] == "Software Marketing") & (train["Course_Type"] == "Course") &
                (train["Long_Promotion"] == 0),"Competition_Metric"]
print(x.shape, x.head())
mean_176 = x.mean(skipna = True)                
print(mean_176)   #0.134          
mode_176 = x.mode()                
print(mode_176) #0.076
median_176 = x.median(skipna = True)
median_176  #0.086



(25908,) 23446    0.082
23447    0.082
23448    0.082
23449    0.082
23450    0.082
Name: Competition_Metric, dtype: float64
0.13359418205066395
0    0.076
dtype: float64


0.086

In [21]:
#finding mean/ mode / median of  all courses with Development, Program and with 0 long promotion for 354


y = train.loc[(train["Course_Domain"] == "Development") & (train["Course_Type"] == "Program") &
                (train["Long_Promotion"] == 0),"Competition_Metric"]
print(y.shape,y.head())
mean_354 = y.mean(skipna = True)                
print(mean_354)   #0.126            
mode_354 = y.mode()                
print(mode_354) #0.002
median_354 = y.median(skipna = True) 
median_354  #0.052

(63136,) 1764    0.316
1765    0.316
1766    0.316
1767    0.316
1768    0.316
Name: Competition_Metric, dtype: float64
0.12552838371834948
0    0.002
dtype: float64


0.052000000000000005

In [22]:
### creating new dataset train_mode by filling competition metric vlues of course id 176 1nd 354 
#with mode_176 and mode_354 

train_mode = train.copy()

for i in train_mode.index:
    #print(i)
    #print(j)
    if train_mode.at[i,"Course_ID"] == 176:
        train_mode.at[i,"Competition_Metric"] = mode_176
    if train_mode.at[i,"Course_ID"] == 354:
        train_mode.at[i,"Competition_Metric"]  = mode_354

print(train_mode.isnull().sum())   
train_mode.head()    
    


ID                    0
Day_No                0
Course_ID             0
Course_Domain         0
Course_Type           0
Short_Promotion       0
Public_Holiday        0
Long_Promotion        0
User_Traffic          0
Competition_Metric    0
Sales                 0
dtype: int64


,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales
0,1,1,1,Development,Course,0,1,1,11004,0.007,81
1,2,2,1,Development,Course,0,0,1,13650,0.007,79
2,3,3,1,Development,Course,0,0,1,11655,0.007,75
3,4,4,1,Development,Course,0,0,1,12054,0.007,80
4,5,5,1,Development,Course,0,0,1,6804,0.007,41


In [0]:
#train_mode.to_csv("train_mode.csv")   

In [23]:
### creating new dataset valid_mode by filling competition metric vlues of course id 176 1nd 354 
#with mode_176 and mode_354

valid_mode = valid.copy()

for i in valid_mode.index:
    #print(i)
    #print(j)
    if valid_mode.at[i,"Course_ID"] == 176:
        valid_mode.at[i,"Competition_Metric"] = mode_176
    if valid_mode.at[i,"Course_ID"] == 354:
        valid_mode.at[i,"Competition_Metric"]  = mode_354

print(valid_mode.isnull().sum())   
print(valid_mode.head())  

valid_mode.shape


ID                    0
Day_No                0
Course_ID             0
Course_Domain         0
Course_Type           0
Short_Promotion       0
Public_Holiday        0
Long_Promotion        0
Competition_Metric    0
dtype: int64
    ID  Day_No Course_ID  ... Public_Holiday Long_Promotion Competition_Metric
0  883     883         1  ...              0              1              0.007
1  884     884         1  ...              0              1              0.007
2  885     885         1  ...              0              1              0.007
3  886     886         1  ...              0              1              0.007
4  887     887         1  ...              0              1              0.007

[5 rows x 9 columns]


(36000, 9)

In [0]:
valid_mode.to_csv("valid_mode.csv") 

In [24]:
### creating new dataset train_mean by filling competition metric vlues of course id 176 1nd 354 
#with mean_176 and mean_354 

train_mean = train.copy()

for i in train_mean.index:
    #print(i)
    #print(j)
    if train_mean.at[i,"Course_ID"] == 176:
        train_mean.at[i,"Competition_Metric"] = mean_176
    if train_mean.at[i,"Course_ID"] == 354:
        train_mean.at[i,"Competition_Metric"]  = mean_354

print(train_mean.isnull().sum())   
train_mean.head()    
    

ID                    0
Day_No                0
Course_ID             0
Course_Domain         0
Course_Type           0
Short_Promotion       0
Public_Holiday        0
Long_Promotion        0
User_Traffic          0
Competition_Metric    0
Sales                 0
dtype: int64


,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales
0,1,1,1,Development,Course,0,1,1,11004,0.007,81
1,2,2,1,Development,Course,0,0,1,13650,0.007,79
2,3,3,1,Development,Course,0,0,1,11655,0.007,75
3,4,4,1,Development,Course,0,0,1,12054,0.007,80
4,5,5,1,Development,Course,0,0,1,6804,0.007,41


In [0]:
#train_mean.to_csv("train_mean.csv")   

In [25]:
### creating new dataset valid_mean by filling competition metric vlues of course id 176 1nd 354 
#with mean_176 and mean_354

valid_mean = valid.copy()

for i in valid_mean.index:
    #print(i)
    #print(j)
    if valid_mean.at[i,"Course_ID"] == 176:
        valid_mean.at[i,"Competition_Metric"] = mean_176
    if valid_mean.at[i,"Course_ID"] == 354:
        valid_mean.at[i,"Competition_Metric"]  = mean_354

print(valid_mean.isnull().sum())   
print(valid_mean.head() )
valid_mean.shape



ID                    0
Day_No                0
Course_ID             0
Course_Domain         0
Course_Type           0
Short_Promotion       0
Public_Holiday        0
Long_Promotion        0
Competition_Metric    0
dtype: int64
    ID  Day_No Course_ID  ... Public_Holiday Long_Promotion Competition_Metric
0  883     883         1  ...              0              1              0.007
1  884     884         1  ...              0              1              0.007
2  885     885         1  ...              0              1              0.007
3  886     886         1  ...              0              1              0.007
4  887     887         1  ...              0              1              0.007

[5 rows x 9 columns]


(36000, 9)

In [0]:
valid_mean.to_csv("valid_mean.csv")

In [0]:
train_mode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512087 entries, 0 to 512086
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   ID                  512087 non-null  int64   
 1   Day_No              512087 non-null  int64   
 2   Course_ID           512087 non-null  category
 3   Course_Domain       512087 non-null  category
 4   Course_Type         512087 non-null  category
 5   Short_Promotion     512087 non-null  category
 6   Public_Holiday      512087 non-null  category
 7   Long_Promotion      512087 non-null  category
 8   Competition_Metric  512087 non-null  float64 
 9   Sales               512087 non-null  int64   
dtypes: category(6), float64(1), int64(3)
memory usage: 19.1 MB


# understanding the data through graphs

In [0]:
train_mode.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,512087,NaN,NaN,NaN,274007,158229,1,136962,273984,411066,548027
Day_No,512087,NaN,NaN,NaN,434.918,256.044,1,214,427,658,882
Course_ID,512087,600,600,882,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Course_Domain,512087,4,Development,264295,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Course_Type,512087,3,Course,262747,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short_Promotion,512087,2,0,317369,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Public_Holiday,512087,2,0,495885,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long_Promotion,512087,2,0,261693,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Competition_Metric,512087,NaN,NaN,NaN,0.0732268,0.0999867,0,0.01,0.035,0.094,0.768
Sales,512087,NaN,NaN,NaN,120.827,54.3553,0,84,111,146,682


In [0]:
valid_mode.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,36000,NaN,NaN,NaN,274566,158084,883,137730,274762,410873,548087
Day_No,36000,NaN,NaN,NaN,912.5,17.3183,883,897.75,912.5,927.25,942
Course_ID,36000,600,600,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Course_Domain,36000,4,Development,18480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Course_Type,36000,3,Course,18600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short_Promotion,36000,2,0,21600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Public_Holiday,36000,2,0,35605,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long_Promotion,36000,2,1,18060,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Competition_Metric,36000,NaN,NaN,NaN,0.072825,0.0992881,0,0.01,0.0345,0.0925,0.768



##number of unique levels are same in both trin and valid and mean of Competition_Metric is also almost same.


In [0]:
colormap = plt.cm.RdBu
plt.figure(figsize=(32,10))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train_mode.corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)
# Note that the categorical features have been neglected in the
# correlation matrix.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

############ there is negative correlation b/w Competition_Metric and sales

In [0]:
sns.distplot(train['Sales'],hist=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#almost sales are in nirmal dustribution with little right skew

In [0]:
sns.distplot(train_mode['Competition_Metric'],hist=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [0]:
sns.distplot(train_mean['Competition_Metric'],hist=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Competition_Metric is not normally distributed


In [0]:
sns.scatterplot(x='Competition_Metric', y='Sales', data = train_mode)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### there is no trend b/w Sales and competition metric but slight trend that if Cometition metric increases, sales decrease


In [0]:

sns.barplot(x="Sales", y="Course_ID", hue="Long_Promotion", data=train_mode, palette="Set1")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [0]:
sns.barplot(x="Course_Domain", y="Sales", hue="Long_Promotion", data=train_mode, palette="Set1")

### long promotion increase sales in business only where as in reamining three long promo decreases sales

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [0]:
sns.barplot(x="Course_Domain", y="Sales", hue="Short_Promotion", data=train_mode, palette="Set1")
#short promotion increases sales in all domain

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [0]:
sns.barplot(x="Course_Domain", y="Sales", hue="Public_Holiday", data=train_mode, palette="Set1")
## public holidays decrease sales except in business domain

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [0]:
sns.barplot(x="Course_Type", y="Sales", hue="Public_Holiday", data=train_mode, palette="Set1")
#sales are more for only Degree if Public holidays are more. 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [0]:
sns.barplot(x="Course_Type", y="Sales", hue="Short_Promotion", data=train_mode, palette="Set1")
## short promotion increases all types of course types

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [0]:
sns.barplot(x="Course_Type", y="Sales", hue="Long_Promotion", data=train_mode, palette="Set1")
## long promotion influences only Degree

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [0]:
## separatig predictors (keep only columns which are useful in predictions)and predicted 

In [0]:
keepcols = ['Course_ID','Course_Domain','Course_Type','Short_Promotion','Public_Holiday','Long_Promotion','Competition_Metric']
train_mode_original = train_mode.copy()      
X_train_mode, y_train_mode = train_mode.loc[:,keepcols], train_mode.loc[:,'Sales']

X_valid_mode = valid_mode.loc[:,keepcols]
print(X_train_mode.shape, y_train_mode.shape)
print(X_valid_mode.shape)

(512087, 7) (512087,)
(36000, 7)


In [26]:
keepcols = ['Course_ID','Course_Domain','Course_Type','Short_Promotion','Public_Holiday','Long_Promotion','Competition_Metric']
train_mean_original = train_mean.copy()     
X_train_mean, y_train_mean = train_mean.loc[:,keepcols], train_mean.loc[:,'Sales']

X_valid_mean = valid_mean.loc[:,keepcols]
print(X_train_mean.shape, y_train_mean.shape)
print(X_valid_mean.shape)

(512087, 7) (512087,)
(36000, 7)


##Train and test data split of train_mode for model building


In [0]:
X_train_mode,X_test_mode,y_train_mode,y_test_mode = train_test_split(X_train_mode,
                                                 y_train_mode,test_size = 0.2,random_state=1)

train_mode = pd.concat([X_train_mode,y_train_mode],axis=1).reset_index().drop(['index'],1)
test_mode = pd.concat([X_test_mode,y_test_mode],axis=1).reset_index().drop(['index'],1)
print("TRAIN DATA SHAPE IS : ",train_mode.shape)
print("TEST DATA SHAPE IS :",test_mode.shape)

TRAIN DATA SHAPE IS :  (409669, 8)
TEST DATA SHAPE IS : (102418, 8)


In [0]:
num_cols

train_mode[num_cols].head

<bound method NDFrame.head of         Competition_Metric  Sales
0                    0.055    158
1                    0.297    174
2                    0.101    111
3                    0.004     61
4                    0.003     78
...                    ...    ...
409664               0.158    123
409665               0.066    152
409666               0.049     70
409667               0.011    169
409668               0.001    181

[409669 rows x 2 columns]>

#scalling numerical variables Sales and Competition metric

In [0]:
train_mode_num = train_mode[num_cols]
test_mode_num = test_mode[num_cols]
valid_mode_num = valid_mode["Competition_Metric"]
print(train_mode_num.head())
print(test_mode_num.head())
print(valid_mode_num.head())

   Competition_Metric  Sales
0               0.055    158
1               0.297    174
2               0.101    111
3               0.004     61
4               0.003     78
   Competition_Metric  Sales
0               0.465    191
1               0.007    160
2               0.005    113
3               0.051    118
4               0.030    104
0    0.007
1    0.007
2    0.007
3    0.007
4    0.007
Name: Competition_Metric, dtype: float64


#stdscale = StandardScaler()

#stdscale.fit(train_mode_num)
#train_mode_num = pd.DataFrame(stdscale.transform(train_mode_num),columns = num_cols)
#print(train_mode_num.head())

##test_mode_num = pd.DataFrame(stdscale.transform(test_mode_num),columns = num_cols)
#print(test_mode_num.head())

#test_mode_scaledactuals = test_mode_num['Sales']
#print(test_mode_scaledactuals.head())


##Dummification

In [0]:
cat_cols = ['Course_ID','Course_Domain', 'Course_Type','Short_Promotion','Public_Holiday','Long_Promotion']
train_mode_cat = train_mode[cat_cols]
test_mode_cat = test_mode[cat_cols]
valid_mode_cat = valid_mode[cat_cols]
train_mode_cat_dummy = pd.get_dummies(train_mode_cat,prefix_sep='_', drop_first=True)
test_mode_cat_dummy = pd.get_dummies(test_mode_cat,prefix_sep='_', drop_first=True)
valid_mode_cat_dummy = pd.get_dummies(valid_mode_cat,prefix_sep='_', drop_first=True)
print(train_mode_cat_dummy.shape)
print(test_mode_cat_dummy.shape)
print(valid_mode_cat_dummy.shape)



(409669, 607)
(102418, 607)
(36000, 607)


In [0]:
train_mode_df = pd.concat((train_mode.loc[:,num_cols],train_mode_cat_dummy),axis=1)
test_mode_df = pd.concat((test_mode.loc[:,num_cols],test_mode_cat_dummy),axis=1)
valid_mode_df = pd.concat((valid_mode.loc[:,'Competition_Metric'],valid_mode_cat_dummy),axis=1)
print(train_mode_df.shape)
print(test_mode_df.shape)
print(valid_mode_df.shape)


(409669, 609)
(102418, 609)
(36000, 608)


In [0]:
print(train_mode_df.head)
print(test_mode_df.head)
print(valid_mode_df.head)


<bound method NDFrame.head of         Competition_Metric  Sales  ...  Public_Holiday_1  Long_Promotion_1
0                    0.055    158  ...                 0                 0
1                    0.297    174  ...                 0                 1
2                    0.101    111  ...                 0                 1
3                    0.004     61  ...                 0                 0
4                    0.003     78  ...                 0                 0
...                    ...    ...  ...               ...               ...
409664               0.158    123  ...                 0                 0
409665               0.066    152  ...                 0                 1
409666               0.049     70  ...                 0                 1
409667               0.011    169  ...                 0                 1
409668               0.001    181  ...                 0                 1

[409669 rows x 609 columns]>
<bound method NDFrame.head of         Co

##after dummification columns number is same in all three (train,test and valid) data sets 
#so there will be no problem in fitting the model

# creating datasets for only label encoding

In [0]:
class MultiColumnLabelEncoder(LabelEncoder):
    """
    Wraps sklearn LabelEncoder functionality for use on multiple columns of a
    pandas dataframe.

    """
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, dframe):
        """
        Fit label encoder to pandas columns.

        Access individual column classes via indexig `self.all_classes_`

        Access individual column encoders via indexing
        `self.all_encoders_`
        """
        # if columns are provided, iterate through and get `classes_`
        if self.columns is not None:
            # ndarray to hold LabelEncoder().classes_ for each
            # column; should match the shape of specified `columns`
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            self.all_encoders_ = np.ndarray(shape=self.columns.shape,
                                            dtype=object)
            for idx, column in enumerate(self.columns):
                # fit LabelEncoder to get `classes_` for the column
                le = LabelEncoder()
                le.fit(dframe.loc[:, column].values)
                # append the `classes_` to our ndarray container
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                # append this column's encoder
                self.all_encoders_[idx] = le
        else:
            # no columns specified; assume all are to be encoded
            self.columns = dframe.iloc[:, :].columns
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            for idx, column in enumerate(self.columns):
                le = LabelEncoder()
                le.fit(dframe.loc[:, column].values)
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                self.all_encoders_[idx] = le
        return self

    def fit_transform(self, dframe):
        """
        Fit label encoder and return encoded labels.

        Access individual column classes via indexing
        `self.all_classes_`

        Access individual column encoders via indexing
        `self.all_encoders_`

        Access individual column encoded labels via indexing
        `self.all_labels_`
        """
        # if columns are provided, iterate through and get `classes_`
        if self.columns is not None:
            # ndarray to hold LabelEncoder().classes_ for each
            # column; should match the shape of specified `columns`
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            self.all_encoders_ = np.ndarray(shape=self.columns.shape,
                                            dtype=object)
            self.all_labels_ = np.ndarray(shape=self.columns.shape,
                                          dtype=object)
            for idx, column in enumerate(self.columns):
                # instantiate LabelEncoder
                le = LabelEncoder()
                # fit and transform labels in the column
                dframe.loc[:, column] =\
                    le.fit_transform(dframe.loc[:, column].values)
                # append the `classes_` to our ndarray container
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                self.all_encoders_[idx] = le
                self.all_labels_[idx] = le
        else:
            # no columns specified; assume all are to be encoded
            self.columns = dframe.iloc[:, :].columns
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            for idx, column in enumerate(self.columns):
                le = LabelEncoder()
                dframe.loc[:, column] = le.fit_transform(
                        dframe.loc[:, column].values)
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                self.all_encoders_[idx] = le
        return dframe.loc[:, self.columns].values

    def transform(self, dframe):
        """
        Transform labels to normalized encoding.
        """
        if self.columns is not None:
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[
                    idx].transform(dframe.loc[:, column].values)
        else:
            self.columns = dframe.iloc[:, :].columns
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[idx]\
                    .transform(dframe.loc[:, column].values)
        return dframe.loc[:, self.columns].values

    def inverse_transform(self, dframe):
        """
        Transform labels back to original encoding.
        """
        if self.columns is not None:
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[idx]\
                    .inverse_transform(dframe.loc[:, column].values)
        else:
            self.columns = dframe.iloc[:, :].columns
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[idx]\
                    .inverse_transform(dframe.loc[:, column].values)
        return dframe.loc[:, self.columns].values

In [0]:
train_mode_cat.columns

Index(['Course_ID', 'Course_Domain', 'Course_Type', 'Short_Promotion',
       'Public_Holiday', 'Long_Promotion'],
      dtype='object')

In [0]:
df_object_columns = train_mode_cat.iloc[:, :].columns
mcle = MultiColumnLabelEncoder(columns=df_object_columns)
mcle.fit(train_mode_cat)
train_mode_le = pd.DataFrame(mcle.transform(train_mode_cat),columns = [cat_cols])
print(train_mode_le.head())
test_mode_le = pd.DataFrame(mcle.fit_transform(test_mode_cat),columns = [cat_cols])
print(test_mode_le.head())
valid_mode_le = pd.DataFrame(mcle.fit_transform(valid_mode_cat),columns = [cat_cols])
print(valid_mode_le.head())

  Course_ID Course_Domain  ... Public_Holiday Long_Promotion
0       542             2  ...              0              0
1        36             3  ...              0              1
2        19             3  ...              0              1
3       123             2  ...              0              0
4       502             1  ...              0              0

[5 rows x 6 columns]
  Course_ID Course_Domain  ... Public_Holiday Long_Promotion
0       414             1  ...              0              0
1       421             3  ...              0              1
2       154             1  ...              0              0
3       524             2  ...              0              0
4       344             1  ...              0              0

[5 rows x 6 columns]
  Course_ID Course_Domain  ... Public_Holiday Long_Promotion
0         0             1  ...              0              1
1         0             1  ...              0              1
2         0             1  ...           

In [0]:
train_mode_lef = pd.concat((train_mode.loc[:,num_cols],train_mode_le),axis=1)
test_mode_lef = pd.concat((test_mode.loc[:,num_cols],test_mode_le),axis=1)
valid_mode_lef = pd.concat((valid_mode.loc[:,'Competition_Metric'],valid_mode_le),axis=1)
print(train_mode_lef.shape)
print(test_mode_lef.shape)
print(valid_mode_lef.shape)


(409669, 8)
(102418, 8)
(36000, 7)


In [0]:
print(train_mode_lef.head)
print(test_mode_lef.head)
print(valid_mode_lef.head)


<bound method NDFrame.head of         Competition_Metric  Sales  ...  (Public_Holiday,)  (Long_Promotion,)
0                    0.055    158  ...                  0                  0
1                    0.297    174  ...                  0                  1
2                    0.101    111  ...                  0                  1
3                    0.004     61  ...                  0                  0
4                    0.003     78  ...                  0                  0
...                    ...    ...  ...                ...                ...
409664               0.158    123  ...                  0                  0
409665               0.066    152  ...                  0                  1
409666               0.049     70  ...                  0                  1
409667               0.011    169  ...                  0                  1
409668               0.001    181  ...                  0                  1

[409669 rows x 8 columns]>
<bound method NDFr

# Model building with label encoded data_ mode

In [0]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks")

from sklearn.preprocessing import LabelEncoder,StandardScaler

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,mean_absolute_error
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from statsmodels.api import Logit
import xgboost as xgb
import lightgbm as lgb



from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.externals.six import StringIO  
from IPython.display import Image  

import warnings
warnings.filterwarnings('ignore')

In [0]:
trainx, trainy = train_mode_lef.loc[:,train_mode_lef.columns!='Sales'], train_mode_lef.loc[:,'Sales']

testx, testy = test_mode_lef.loc[:,test_mode_lef.columns!='Sales'], test_mode_lef.loc[:,'Sales']

In [0]:
print(trainx.shape)
print(trainy.shape)
print(testx.shape)
print(testy.shape)

(409669, 7)
(409669,)
(102418, 7)
(102418,)


In [0]:
##xgb.XGBRegressor

In [0]:
model = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.5, learning_rate=0.2, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=10, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, metric = '1000*rmsle',
       silent=True, subsample=1)
model.fit(X = trainx, y = trainy)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.5,
             importance_type='gain', learning_rate=0.2, max_delta_step=0,
             max_depth=5, metric='1000*rmsle', min_child_weight=1, missing=None,
             n_estimators=100, n_jobs=1, nthread=10, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=True, subsample=1, verbosity=1)

In [0]:
import time
start_time = time.time()
y_train_pred_xgbm = model.predict(trainx)
y_test_pred_xgbm = model.predict(testx)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.6873583793640137 seconds ---


In [0]:
trainpredict = pd.DataFrame(y_train_pred_xgbm)
testpredict = pd.DataFrame(y_test_pred_xgbm)

In [0]:

from sklearn.metrics import mean_squared_log_error 
from math import sqrt
rmsle_train = 1000*(sqrt(mean_squared_log_error(trainy, trainpredict)))


rmsle_test = 1000*(sqrt(mean_squared_log_error(testy, testpredict)))   

    
print(rmsle_train)
print(rmsle_test)

293.8002655207236
295.14758526967313


In [0]:
#Predict 
valid_predict_mode_xgbm_lef = model.predict(data=valid_mode_lef)
valid_predict_mode_xgbm_lef = pd.DataFrame(valid_predict_mode_xgbm_lef)
valid_predict_mode_xgbm_lef.to_csv("valid_predict_mode_xgbm_lef.csv")
valid_predict_mode_fulldata_xgbm_lef = pd.concat((valid_mode_lef,valid_predict_mode_xgbm_lef),axis=1)
valid_predict_mode_fulldata_xgbm_lef.to_csv("valid_predict_mode_fulldata_xgbm_lef.csv")

# Model building with label encoded data_ mean

In [0]:
##Train and test data split of train_mean for model building

In [28]:
X_train_mean,X_test_mean,y_train_mean,y_test_mean = train_test_split(X_train_mean,
                                                 y_train_mean,test_size = 0.2,random_state=1)

train_mean = pd.concat([X_train_mean,y_train_mean],axis=1).reset_index().drop(['index'],1)
test_mean = pd.concat([X_test_mean,y_test_mean],axis=1).reset_index().drop(['index'],1)
print("TRAIN DATA SHAPE IS : ",train_mean.shape)
print("TEST DATA SHAPE IS :",test_mean.shape)

TRAIN DATA SHAPE IS :  (409669, 8)
TEST DATA SHAPE IS : (102418, 8)


In [29]:
train_mean_num = train_mean[num_cols]
test_mean_num = test_mean[num_cols]
valid_mean_num = valid_mean["Competition_Metric"]
print(train_mean_num.head())
print(test_mean_num.head())
print(valid_mean_num.head())

   Competition_Metric  Sales
0               0.055    158
1               0.297    174
2               0.101    111
3               0.004     61
4               0.003     78
   Competition_Metric  Sales
0               0.465    191
1               0.007    160
2               0.005    113
3               0.051    118
4               0.030    104
0    0.007
1    0.007
2    0.007
3    0.007
4    0.007
Name: Competition_Metric, dtype: float64


In [30]:
cat_cols = ['Course_ID','Course_Domain', 'Course_Type','Short_Promotion','Public_Holiday','Long_Promotion']
train_mean_cat = train_mean[cat_cols]
test_mean_cat = test_mean[cat_cols]
valid_mean_cat = valid_mean[cat_cols]
train_mean_cat_dummy = pd.get_dummies(train_mean_cat,prefix_sep='_', drop_first=True)
test_mean_cat_dummy = pd.get_dummies(test_mean_cat,prefix_sep='_', drop_first=True)
valid_mean_cat_dummy = pd.get_dummies(valid_mean_cat,prefix_sep='_', drop_first=True)
print(train_mean_cat_dummy.shape)
print(test_mean_cat_dummy.shape)
print(valid_mean_cat_dummy.shape)

(409669, 607)
(102418, 607)
(36000, 607)


In [32]:
train_mean_df = pd.concat((train_mean.loc[:,num_cols],train_mean_cat_dummy),axis=1)
test_mean_df = pd.concat((test_mean.loc[:,num_cols],test_mean_cat_dummy),axis=1)
valid_mean_df = pd.concat((valid_mean.loc[:,'Competition_Metric'],valid_mean_cat_dummy),axis=1)
print(train_mean_df.shape)
print(test_mean_df.shape)
print(valid_mean_df.shape)


(409669, 609)
(102418, 609)
(36000, 608)


In [33]:
print(train_mean_df.head)
print(test_mean_df.head)
print(valid_mean_df.head)

<bound method NDFrame.head of         Competition_Metric  Sales  ...  Public_Holiday_1  Long_Promotion_1
0                    0.055    158  ...                 0                 0
1                    0.297    174  ...                 0                 1
2                    0.101    111  ...                 0                 1
3                    0.004     61  ...                 0                 0
4                    0.003     78  ...                 0                 0
...                    ...    ...  ...               ...               ...
409664               0.158    123  ...                 0                 0
409665               0.066    152  ...                 0                 1
409666               0.049     70  ...                 0                 1
409667               0.011    169  ...                 0                 1
409668               0.001    181  ...                 0                 1

[409669 rows x 609 columns]>
<bound method NDFrame.head of         Co

In [34]:
df_object_columns = train_mean_cat.iloc[:, :].columns
mcle = MultiColumnLabelEncoder(columns=df_object_columns)
mcle.fit(train_mean_cat)
train_mean_le = pd.DataFrame(mcle.transform(train_mean_cat),columns = [cat_cols])
print(train_mean_le.head())
test_mean_le = pd.DataFrame(mcle.fit_transform(test_mean_cat),columns = [cat_cols])
print(test_mean_le.head())
valid_mean_le = pd.DataFrame(mcle.fit_transform(valid_mean_cat),columns = [cat_cols])
print(valid_mean_le.head())

  Course_ID Course_Domain  ... Public_Holiday Long_Promotion
0       542             2  ...              0              0
1        36             3  ...              0              1
2        19             3  ...              0              1
3       123             2  ...              0              0
4       502             1  ...              0              0

[5 rows x 6 columns]
  Course_ID Course_Domain  ... Public_Holiday Long_Promotion
0       414             1  ...              0              0
1       421             3  ...              0              1
2       154             1  ...              0              0
3       524             2  ...              0              0
4       344             1  ...              0              0

[5 rows x 6 columns]
  Course_ID Course_Domain  ... Public_Holiday Long_Promotion
0         0             1  ...              0              1
1         0             1  ...              0              1
2         0             1  ...           

In [35]:
train_mean_lef = pd.concat((train_mean.loc[:,num_cols],train_mean_le),axis=1)
test_mean_lef = pd.concat((test_mean.loc[:,num_cols],test_mean_le),axis=1)
valid_mean_lef = pd.concat((valid_mean.loc[:,'Competition_Metric'],valid_mean_le),axis=1)
print(train_mean_lef.shape)
print(test_mean_lef.shape)
print(valid_mean_lef.shape)

(409669, 8)
(102418, 8)
(36000, 7)


In [36]:
print(train_mean_lef.head)
print(test_mean_lef.head)
print(valid_mean_lef.head)

<bound method NDFrame.head of         Competition_Metric  Sales  ...  (Public_Holiday,)  (Long_Promotion,)
0                    0.055    158  ...                  0                  0
1                    0.297    174  ...                  0                  1
2                    0.101    111  ...                  0                  1
3                    0.004     61  ...                  0                  0
4                    0.003     78  ...                  0                  0
...                    ...    ...  ...                ...                ...
409664               0.158    123  ...                  0                  0
409665               0.066    152  ...                  0                  1
409666               0.049     70  ...                  0                  1
409667               0.011    169  ...                  0                  1
409668               0.001    181  ...                  0                  1

[409669 rows x 8 columns]>
<bound method NDFr

# Model building with label encoded data_mean

In [0]:
trainx, trainy = train_mean_lef.loc[:,train_mean_lef.columns!='Sales'], train_mean_lef.loc[:,'Sales']

testx, testy = test_mean_lef.loc[:,test_mean_lef.columns!='Sales'], test_mean_lef.loc[:,'Sales']

In [38]:
print(trainx.shape)
print(trainy.shape)
print(testx.shape)
print(testy.shape)

(409669, 7)
(409669,)
(102418, 7)
(102418,)


In [0]:
model = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.5, learning_rate=0.2, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=10, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, metric = '1000*rmsle',
       silent=True, subsample=1)
model.fit(X = trainx, y = trainy)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.5,
             importance_type='gain', learning_rate=0.2, max_delta_step=0,
             max_depth=5, metric='1000*rmsle', min_child_weight=1, missing=None,
             n_estimators=100, n_jobs=1, nthread=10, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=True, subsample=1, verbosity=1)

In [0]:
import time
start_time = time.time()
y_train_pred_xgbm = model.predict(trainx)
y_test_pred_xgbm = model.predict(testx)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.7438251972198486 seconds ---


In [0]:
trainpredict = pd.DataFrame(y_train_pred_xgbm)
testpredict = pd.DataFrame(y_test_pred_xgbm)

In [0]:
from sklearn.metrics import mean_squared_log_error 
from math import sqrt
rmsle_train = 1000*(sqrt(mean_squared_log_error(trainy, trainpredict)))


rmsle_test = 1000*(sqrt(mean_squared_log_error(testy, testpredict)))   

    
print(rmsle_train)
print(rmsle_test)

292.2199010380627
293.308619275427


In [0]:
#Predict 
valid_predict_mean_xgbm_lef = model.predict(data=valid_mean_lef)
valid_predict_mean_xgbm_lef = pd.DataFrame(valid_predict_mean_xgbm_lef)
valid_predict_mean_xgbm_lef.to_csv("valid_predict_mean_xgbm_lef.csv")
valid_predict_mean_fulldata_xgbm_lef = pd.concat((valid_mean_lef,valid_predict_mean_xgbm_lef),axis=1)
valid_predict_mean_fulldata_xgbm_lef.to_csv("valid_predict_mean_fulldata_xgbm_lef.csv")

In [0]:
### HyperOpt Parameter Tuning
from hyperopt import tpe
from hyperopt import STATUS_OK
from hyperopt import Trials
from hyperopt import hp
from hyperopt import fmin
from functools import partial

N_FOLDS = 10
MAX_EVALS = 50

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer

def gini(truth, predictions):
    print('truth is {}  and predictions is {}',truth)
    g = np.asarray(np.c_[truth, predictions, np.arange(len(truth)) ], dtype=np.float)
    print('Computed values is {}  ',g)
    print('g of 2 is ', g[:,2])
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(truth) + 1) / 2.
    return gs / len(truth)

def gini_sklearn(truth, predictions):
    return gini(truth, predictions) / gini(truth, truth)

gini_scorer = make_scorer(gini_sklearn, greater_is_better = False)

    
#def objective_randomForest(params):
def objective_XGBRegressor(params):
    params = {'n_estimators': int(params['n_estimators']), 'max_depth': int(params['max_depth'])}
    clf_rf = xgb.XGBRegressor(n_jobs=4, **params)
    score = cross_val_score(clf_rf, trainx, trainy, scoring=gini_scorer, cv=StratifiedKFold()).mean()
    print("Gini {:.3f} params {}".format(score, params))
    return score

space = {
    'n_estimators': hp.quniform('n_estimators', 25, 300, 25),
    'max_depth': hp.quniform('max_depth', 1, 12, 1)
}

best = fmin(fn=objective_XGBRegressor,
            space=space,
            algo=tpe.suggest,
            max_evals=70)

[18:54:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
truth is {}  and predictions is {}
0         158
1         174
2         111
3          61
4          78
         ... 
268059    460
305626    564
355258    468
384363    482
392916    499
Name: Sales, Length: 81934, dtype: int64
Computed values is {}  
[[1.58000000e+02 1.46642410e+02 0.00000000e+00]
 [1.74000000e+02 1.61706635e+02 1.00000000e+00]
 [1.11000000e+02 1.43068268e+02 2.00000000e+00]
 ...
 [4.68000000e+02 3.06943481e+02 8.19310000e+04]
 [4.82000000e+02 2.34165298e+02 8.19320000e+04]
 [4.99000000e+02 3.06943481e+02 8.19330000e+04]]
g of 2 is 
[0.0000e+00 1.0000e+00 2.0000e+00 ... 8.1931e+04 8.1932e+04 8.1933e+04]
truth is {}  and predictions is {}
0         158
1         174
2         111
3          61
4          78
         ... 
268059    460
305626    564
355258    468
384363    482
392916    499
Name: Sales, Length: 81934, dtype: int64
Computed val

In [41]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
sns.set()
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import statsmodels.api as sm
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split

import warnings
# import the_module_that_warns

warnings.filterwarnings("ignore")

In [47]:
%%time

def model(train_x,train_y,test_x,test_y,col):
    params = {
        'nthread': 10,
         'max_depth': 5,
#         'max_depth': 9,
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression_l1',
        'metric': 'rmse', # this is abs(a-e)/max(1,a)
#         'num_leaves': 39,
        'num_leaves': 64,
        'learning_rate': 0.2,
       'feature_fraction': 0.9,
#         'feature_fraction': 0.8108472661400657,
#         'bagging_fraction': 0.9837558288375402,
       'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'lambda_l1': 3.097758978478437,
        'lambda_l2': 2.9482537987198496,
#       'lambda_l1': 0.06,
#       'lambda_l2': 0.1,
        'verbose': 1,
        'min_child_weight': 6.996211413900573,
        'min_split_gain': 0.037310344962162616,
        }
    
    lgb_train = lgb.Dataset(train_x,train_y)
    lgb_valid = lgb.Dataset(test_x,test_y)
    model = lgb.train(params, lgb_train, 3000, valid_sets=[lgb_train, lgb_valid],early_stopping_rounds=50, verbose_eval=50)
    return model

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.39 µs


In [48]:
%%time
col = ['Course_ID', 'Course_Domain', 'Course_Type', 'Short_Promotion',
       'Public_Holiday', 'Long_Promotion','Competition_Metric']
      
model = model(trainx,trainy,testx,testy,col)

Training until validation scores don't improve for 50 rounds.
[50]	training's rmse: 39.7284	valid_1's rmse: 39.8749
[100]	training's rmse: 33.1928	valid_1's rmse: 33.3916
[150]	training's rmse: 30.6721	valid_1's rmse: 30.8592
[200]	training's rmse: 29.8644	valid_1's rmse: 30.0781
[250]	training's rmse: 29.3229	valid_1's rmse: 29.5359
[300]	training's rmse: 29.0599	valid_1's rmse: 29.2638
[350]	training's rmse: 28.8778	valid_1's rmse: 29.0874
[400]	training's rmse: 28.7673	valid_1's rmse: 28.9774
[450]	training's rmse: 28.7041	valid_1's rmse: 28.9125
[500]	training's rmse: 28.6463	valid_1's rmse: 28.8546
[550]	training's rmse: 28.6051	valid_1's rmse: 28.8174
[600]	training's rmse: 28.5835	valid_1's rmse: 28.7977
[650]	training's rmse: 28.568	valid_1's rmse: 28.7842
[700]	training's rmse: 28.5533	valid_1's rmse: 28.7765
[750]	training's rmse: 28.5313	valid_1's rmse: 28.7563
[800]	training's rmse: 28.5241	valid_1's rmse: 28.7512
[850]	training's rmse: 28.5037	valid_1's rmse: 28.7283
[900]

In [49]:
model

In [50]:
import time
start_time = time.time()
y_train_pred_mean_lgbm = model.predict(trainx)
y_test_pred_mean_lgbm = model.predict(testx)
print("--- %s seconds ---" % (time.time() - start_time))

--- 37.026774883270264 seconds ---


In [0]:
trainpredict = pd.DataFrame(y_train_pred_mean_lgbm)
testpredict = pd.DataFrame(y_test_pred_mean_lgbm)

In [52]:
from sklearn.metrics import mean_squared_log_error 
from math import sqrt
rmsle_train = 1000*(sqrt(mean_squared_log_error(trainy, trainpredict)))


rmsle_test = 1000*(sqrt(mean_squared_log_error(testy, testpredict)))   

    
print(rmsle_train)
print(rmsle_test)

271.1841990727718
273.0637166815261


In [0]:
#Predict 
valid_predict_mean_lgbm_lef = model.predict(data=valid_mean_lef)
valid_predict_mean_lgbm_lef = pd.DataFrame(valid_predict_mean_lgbm_lef)
valid_predict_mean_lgbm_lef.to_csv("valid_predict_mean_lgbm_lef.csv")
valid_predict_mean_fulldata_lgbm_lef = pd.concat((valid_mean_lef,valid_predict_mean_lgbm_lef),axis=1)
valid_predict_mean_fulldata_lgbm_lef.to_csv("valid_predict_mean_fulldata_lgbm_lef.csv")